In [ ]:
import os

from joblib import delayed

from rule.count_SAQ import CountSAQGenerator
from rule.count_TFQ import CountCompareTFQGenerator
from rule.distance_SAQ import DistanceSAQGenerator
from rule.distance_TFQ import DistanceCompareTFQGenerator
from rule.volume_SAQ import VolumeSAQGenerator
from rule.volume_TFQ import VolumeCompareTFQGenerator
from utils.io import enum_files
from utils.parallel import ParallelTqdm

In [ ]:
def generate_questions(
        generator_class,
        scene_stat_json_file: str,
        output_json_file: str,
        n_questions: int,
):
    generator_class(
        scene_stat_json_file=scene_stat_json_file,
        output_json_file=output_json_file,
    ).generate(n_questions=n_questions)

In [ ]:
export_path = '../output'
scene_json_dir = '../NUMINA-dataset/ScanNet-scene_stats'

tasks = [
    (generator, scene_json, os.path.join(export_path, filename), 6)
    for scene_json in enum_files(scene_json_dir)
    for generator, filename in [
        (CountSAQGenerator, 'NUM-count-SAQ.json'),
        (VolumeSAQGenerator, 'NUM-volume-SAQ.json'),
        (DistanceSAQGenerator, 'NUM-distance-SAQ.json'),
        (CountCompareTFQGenerator, 'NUM-count_compare-TFQ.json'),
        (VolumeCompareTFQGenerator, 'NUM-volume_compare-TFQ.json'),
        (DistanceCompareTFQGenerator, 'NUM-distance_compare-TFQ.json'),
    ]
]

ParallelTqdm(n_jobs=8)(
    delayed(generate_questions)(generator_class, scene_json, output_json, n_questions)
    for generator_class, scene_json, output_json, n_questions in tasks[::5]
)
